In [41]:
import numpy as np
import pandas as pd
import json

user_score = pd.read_csv("DAM_amazon_modle_user_bundle_score.csv")
user_score = user_score.rename({"Unnamed: 0": "user_id"}, axis=1)
user_score = user_score.set_index('user_id')
user_score_matrix = user_score.values
user_score_matrix.shape 
#user_score 
# 1737 (bundle size)

(129, 1737)

In [42]:
user_score[user_score < 1] = 0
user_score

,0,1,2,3,4,5,6,7,8,9,...,1727,1728,1729,1730,1731,1732,1733,1734,1735,1736
user_id,,,,,,,,,,,,,,,,,,,,,
563,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
805,1.811995,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
832,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
829,2.887429,0.0,0.0,0.0,0.0,0.0,3.597836,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
718,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
263,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
834,1.399553,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
import torch
loaded_data = torch.load('amazon_test.pt')

# 提取每个 Tensor 中的值，并保存到列表中
value_list = []
for key in loaded_data.keys():
    tensor = loaded_data[key]
    value = tensor.numpy()
    # 将值添加到列表中
    value_list.append(value)

# 将列表中的所有值组合成一个大的二维数组
true_interaction = np.stack(value_list)


In [44]:

train_data = torch.load('amazon_train.pt')

# 提取每个 Tensor 中的值，并保存到列表中
value_list = []
for key in train_data.keys():
    tensor = train_data[key]
    value = tensor.numpy()
    # 将值添加到列表中
    value_list.append(value)

# 将列表中的所有值组合成一个大的二维数组
train_interaction = np.stack(value_list)

train_interaction.shape


(129, 1737)

In [45]:
#预测的分数中减去一个极大值乘以训练集中的交互记录
large_number = 1e8  # 定义一个较大的数
user_score_matrix = user_score_matrix
user_score_matrix


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.81199479, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.3995527 , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [2.02589297, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [46]:
"""
def ndcg(interactions, predictions, k):
    ndcg_sum = 0.0
    num_users = interactions.shape[0]
    for i in range(num_users):
        # 获取第 i 个用户的预测评分值和交互矩阵
        user_preds = predictions[i]
        user_interactions = interactions[i]

        # 根据预测评分值从高到低排序，并记录原始索引
        sorted_indices = np.argsort(user_preds)[::-1]

        # 计算 DCG 和 IDCG 值
        dcg = 0.0
        idcg = 0.0
        for j in range(k):
            if j >= len(user_interactions):
                break
            rel_i = 1 if user_interactions[sorted_indices[j]] > 0 else 0
            dcg += rel_i / np.log2(j+2)
            if j == 0:
                idcg += 1
            else:
                rel_j = 1 if user_interactions[sorted_indices[j-1]] > 0 else 0
                idcg += rel_j / np.log2(j+2)

        # 计算 NDCG 值并加入总和
        if idcg > 0:
            ndcg_sum += dcg / idcg

    # 计算平均 NDCG 值并返回
    return ndcg_sum / num_users
"""

'\ndef ndcg(interactions, predictions, k):\n    ndcg_sum = 0.0\n    num_users = interactions.shape[0]\n    for i in range(num_users):\n        # 获取第 i 个用户的预测评分值和交互矩阵\n        user_preds = predictions[i]\n        user_interactions = interactions[i]\n\n        # 根据预测评分值从高到低排序，并记录原始索引\n        sorted_indices = np.argsort(user_preds)[::-1]\n\n        # 计算 DCG 和 IDCG 值\n        dcg = 0.0\n        idcg = 0.0\n        for j in range(k):\n            if j >= len(user_interactions):\n                break\n            rel_i = 1 if user_interactions[sorted_indices[j]] > 0 else 0\n            dcg += rel_i / np.log2(j+2)\n            if j == 0:\n                idcg += 1\n            else:\n                rel_j = 1 if user_interactions[sorted_indices[j-1]] > 0 else 0\n                idcg += rel_j / np.log2(j+2)\n\n        # 计算 NDCG 值并加入总和\n        if idcg > 0:\n            ndcg_sum += dcg / idcg\n\n    # 计算平均 NDCG 值并返回\n    return ndcg_sum / num_users\n'

In [47]:
_is_hit_cache = {}

def get_is_hit(scores, ground_truth, topk):
    global _is_hit_cache
    cacheid = (id(scores), id(ground_truth))
    if topk in _is_hit_cache and _is_hit_cache[topk]['id'] == cacheid:
        return _is_hit_cache[topk]['is_hit']
    else:
        device = scores.device
        _, col_indice = torch.topk(scores, topk)
        row_indice = torch.zeros_like(col_indice) + torch.arange(
            scores.shape[0], device=device, dtype=torch.long).view(-1, 1)
        is_hit = ground_truth[row_indice.view(-1),
                              col_indice.view(-1)].view(-1, topk)
        _is_hit_cache[topk] = {'id': cacheid, 'is_hit': is_hit}
        return is_hit


class _Metric:
    '''
    base class of metrics like Recall@k NDCG@k MRR@k
    '''

    def __init__(self):
        self.start()

    @property
    def metric(self):
        return self._metric

    def __call__(self, scores, ground_truth):
        '''
        - scores: model output
        - ground_truth: one-hot test dataset shape=(users, all_bundles/all_items).
        '''
        raise NotImplementedError

    def get_title(self):
        raise NotImplementedError

    def start(self):
        '''
        clear all
        '''
        global _is_hit_cache
        _is_hit_cache = {}
        self._cnt = 0
        self._metric = 0
        self._sum = 0

    def stop(self):
        global _is_hit_cache
        _is_hit_cache = {}
        self._metric = self._sum/self._cnt

class Recall(_Metric):
    '''
    Recall in top-k samples
    '''

    def __init__(self, topk):
        super().__init__()
        self.topk = topk
        self.epison = 1e-8

    def get_title(self):
        return "Recall@{}".format(self.topk)

    def __call__(self, scores, ground_truth):
        is_hit = get_is_hit(scores, ground_truth, self.topk)
        is_hit = is_hit.sum(dim=1)
        num_pos = ground_truth.sum(dim=1)
        self._cnt += scores.shape[0] - (num_pos == 0).sum().item()
        self._sum += (is_hit/(num_pos+self.epison)).sum().item()
        self._metric = self._sum/self._cnt
        return self._metric

class NDCG(_Metric):
    '''
    NDCG in top-k samples
    In this work, NDCG = log(2)/log(1+hit_positions)
    '''

    def DCG(self, hit, device=torch.device('cpu')):
        hit = hit/torch.log2(torch.arange(2, self.topk+2,
                                          device=device, dtype=torch.float))
        return hit.sum(-1)

    def IDCG(self, num_pos):
        hit = torch.zeros(self.topk, dtype=torch.float,device=torch.device('cpu'))
        hit[:num_pos] = 1
        return self.DCG(hit)

    def __init__(self, topk):
        super().__init__()
        self.topk = topk
        self.IDCGs = torch.empty(1 + self.topk, dtype=torch.float,device=torch.device('cpu'))
        self.IDCGs[0] = 1  # avoid 0/0
        for i in range(1, self.topk + 1):
            self.IDCGs[i] = self.IDCG(i)

    def get_title(self):
        return "NDCG@{}".format(self.topk)

    def __call__(self, scores, ground_truth):
        device = scores.device
        is_hit = get_is_hit(scores, ground_truth, self.topk)
        num_pos = ground_truth.sum(dim=1).clamp(0, self.topk).to(torch.long)
        dcg = self.DCG(is_hit, device)
        idcg = self.IDCGs[num_pos]
        ndcg = dcg/idcg.to(device)
        self._cnt += scores.shape[0] - (num_pos == 0).sum().item()
        self._sum += ndcg.sum().item()
        self._metric = self._sum/self._cnt
        return self._metric
        

In [48]:
Recall(20), NDCG(20), Recall(40), NDCG(40), Recall(80), NDCG(80)  

(<__main__.Recall at 0x10e6fc430>,
 <__main__.NDCG at 0x1282f9d90>)

In [49]:
5,8,10
user_score_matrix = torch.from_numpy(user_score_matrix)
true_interaction = torch.from_numpy(true_interaction)
print(NDCG.__call__(NDCG(20),user_score_matrix, true_interaction))
print(NDCG.__call__(NDCG(40),user_score_matrix, true_interaction))
print(NDCG.__call__(NDCG(80),user_score_matrix, true_interaction))

0.0017648856769236483
0.0017648856769236483
0.0017648856769236483


In [50]:
5,8,10
print(Recall.__call__(Recall(5),user_score_matrix, true_interaction))
print(Recall.__call__(Recall(8),user_score_matrix, true_interaction))
print(Recall.__call__(Recall(10),user_score_matrix, true_interaction))

0.0
0.0
0.0


In [51]:
# 1,2,3,4
print(Recall.__call__(Recall(1),user_score_matrix, true_interaction))
print(Recall.__call__(Recall(2),user_score_matrix, true_interaction))
print(Recall.__call__(Recall(3),user_score_matrix, true_interaction))
print(Recall.__call__(Recall(4),user_score_matrix, true_interaction))

0.0
0.0
0.0
0.0


In [52]:
# 1,2,3,4
print(NDCG.__call__(NDCG(1),user_score_matrix, true_interaction))
print(NDCG.__call__(NDCG(2),user_score_matrix, true_interaction))
print(NDCG.__call__(NDCG(3),user_score_matrix, true_interaction))
print(NDCG.__call__(NDCG(4),user_score_matrix, true_interaction))

0.0
0.0
0.0
0.0


In [53]:
true_interaction
num_ones = true_interaction.sum()

# 使用 numel() 函数计算 tensor 中所有元素的个数
num_elements = true_interaction.numel()
num_ones

tensor(131.)

In [54]:
num_ones = true_interaction.sum()
num_ones

tensor(131.)